# Example of running Source with changed parameters

We're going to change the impervious fraction parameter in Sacramento and run Source with the original and changed values

In [1]:
# Some steps required until these Python modules are properly installed...
import sys
sys.path.append('../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
import gbr

In [2]:
# Point the system at a particular output directory...
gbr.init('D:/Xike\bu/results/')

In [3]:
# Initialise the Veneer (Source scripting tool)
v = gbr.veneer()

In [9]:
# A path to the variable of interest... .* means 'give me all'
accessor = 'scenario.Network.GetCatchments().*FunctionalUnits.*rainfallRunoffModel.theBaseRRModel.Pctim'

In [18]:
existingValues = v.model.get('scenario.Network.GetCatchments().*FunctionalUnits.*rainfallRunoffModel.lagUHmodifier')
#existingValues = v.model.get('scenario.Network.GetCatchments().*FunctionalUnits.*rainfallRunoffModel.Lztwm')
print(existingValues[:10])

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [11]:
# Run with those original values

# First, set the name of the run
v.model.set('scenario.CurrentConfiguration.runName','RUN_ORIGINAL_PCTIM')

Exception: {'ExceptionType': 'IronPython.Runtime.UnboundNameException', 'InnerException': None, 'StackTrace': '   at IronPython.Runtime.Operations.PythonOps.GetVariable(CodeContext context, String name, Boolean isGlobal, Boolean lightThrow)\r\n   at IronPython.Compiler.LookupGlobalInstruction.Run(InterpretedFrame frame)\r\n   at Microsoft.Scripting.Interpreter.Interpreter.Run(InterpretedFrame frame)\r\n   at Microsoft.Scripting.Interpreter.LightLambda.Run2[T0,T1,TRet](T0 arg0, T1 arg1)\r\n   at IronPython.Compiler.PythonScriptCode.RunWorker(CodeContext ctx)\r\n   at Microsoft.Scripting.Hosting.ScriptSource.Execute(ScriptScope scope)\r\n   at Microsoft.Scripting.Hosting.ScriptSource.Execute[T](ScriptScope scope)\r\n   at FlowMatters.Source.Veneer.RemoteScripting.ScriptRunner.Run(IronPythonScript script) in d:\\src\\projects\\Veneer\\RemoteScripting\\ScriptRunner.cs:line 48', 'Message': "name 'RUN_ORIGINAL_PCTIM' is not defined"}

In [7]:
# Its a good idea to set some options in Dynamic Sednet to prevent the results window appearing
# Also, to make it automatically override existing results
v.configureOptions({'ShowResultsAfterRun':False,'OverwriteResults':True})

{'Exception': None, 'Response': None, 'StandardError': '', 'StandardOut': ''}

In [8]:
# Also, lets switch on the performance options
v.configureOptions({'RunNetworksInParallel':True,'PreRunCatchments':True,'ParallelFlowPhase':True})
v.model.sourceScenarioOptions("PerformanceConfiguration","ProcessCatchmentsInParallel",True)


{'Exception': None,
 'Response': {'Value': True,
  '__type': 'BooleanResponse:#FlowMatters.Source.Veneer.ExchangeObjects'},
 'StandardError': '',
 'StandardOut': ''}

In [9]:
# Now, lets run the model... When this cell executes in Python, the run window should appear in Source...
v.run_model()

(302, 'runs/1')

In [11]:
# NOTE: The above output (eg runs/1) is a point to retrieving the 'normal' Source results - ie all the time series recorders...
# We don't need that for GBR/Dynamic Sednet, because we can get to the summarised results

In [12]:
# Lets take a quick look at those results...
results_original = gbr.Results('RUN_ORIGINAL_PCTIM')
results_original.queries.regional_export('t/y')

SummaryRegion,agbot,agmid,agtop
Constituent,,,
Ametryn,0.000000e+00,0.000000e+00,0.000000e+00
Atrazine,0.000000e+00,0.000000e+00,0.000000e+00
Flow,5.618641e+08,4.826120e+08,2.044276e+08
N_DIN,1.076096e+05,9.206070e+04,3.776663e+04
N_DON,1.076096e+05,9.206070e+04,3.776663e+04
N_Particulate,5.590488e+05,3.949711e+05,2.687237e+05
P_DOP,9.987315e+04,8.674313e+04,3.633789e+04
P_FRP,9.987315e+04,8.674313e+04,3.633789e+04
P_Particulate,NaN,2.748961e+05,1.606461e+05


In [14]:
# NOW... Lets change the Pctim parameter...
# We'll use the same accessor -- but this time we'll use it to set values
accessor

'scenario.Network.GetCatchments().*FunctionalUnits.*rainfallRunoffModel.theBaseRRModel.Pctim'

In [16]:
# We can set every 'instance' of Pctim - ie every FU in every subcatchment - to a single value, with
#
# v.model.set(accessor,0.5)
#
# or we can pass in a list of values
#
# v.model.set(accessor,[0.2,0.3,0.5,0.4,1.0],fromList=True)
#
# Now... If your list of values is shorter than the number of instances... (ie # subcatchments x # FUs),
# then the list will be 'recycled'... That is, the list will be reused repeatedly until values have been assigned to all
# instances...
#
# ie... Given that the Becker's model has 5 FUs, [0.2,0.3,0.5,0.4,1.0] is the equivalent of giving those five values to the
# five functional units in each subcatchment...
v.model.set(accessor,[0.2,0.3,0.5,0.4,1.0],fromList=True)

In [19]:
# Lets check what happened...
v.model.get(accessor)

[0.2,
 0.3,
 0.5,
 0.4,
 1,
 0.2,
 0.3,
 0.5,
 0.4,
 1,
 0.2,
 0.3,
 0.5,
 0.4,
 1,
 0.2,
 0.3,
 0.5,
 0.4,
 1,
 0.2,
 0.3,
 0.5,
 0.4,
 1]

In [20]:
v.model.set('scenario.CurrentConfiguration.runName','RUN_CHANGED_PCTIM')

In [21]:
v.run_model()

(302, 'runs/2')

In [22]:
results_changed = gbr.Results('RUN_CHANGED_PCTIM')
results_changed.queries.regional_export('t/y')

SummaryRegion,agbot,agmid,agtop
Constituent,,,
Ametryn,0.000000e+00,0.000000e+00,0.000000e+00
Atrazine,0.000000e+00,0.000000e+00,0.000000e+00
Flow,5.985996e+09,5.482349e+09,5.046765e+09
N_DIN,1.318299e+06,1.218539e+06,9.657002e+05
N_DON,1.318299e+06,1.218539e+06,9.657002e+05
N_Particulate,2.604056e+07,2.483552e+07,3.695302e+07
P_DOP,1.265883e+06,1.177535e+06,9.466321e+05
P_FRP,1.265883e+06,1.177535e+06,9.466321e+05
P_Particulate,1.114083e+11,1.114078e+11,1.370545e+11


In [24]:
# Now that we've done both runs, we probably want to put the parameter back to normal...
v.model.set(accessor,existingValues,fromList=True)


In [25]:
# Now... Lets run a results comparison...
differences = gbr.DifferenceResults('RUN_ORIGINAL_PCTIM','RUN_CHANGED_PCTIM')
differences.queries.regional_export('t/y')

SummaryRegion,agbot,agmid,agtop
Constituent,,,
Ametryn,0.000000e+00,0.000000e+00,0.000000e+00
Atrazine,0.000000e+00,0.000000e+00,0.000000e+00
Flow,-5.424132e+09,-4.999737e+09,-4.842337e+09
N_DIN,-1.210689e+06,-1.126478e+06,-9.279335e+05
N_DON,-1.210689e+06,-1.126478e+06,-9.279335e+05
N_Particulate,-2.548151e+07,-2.444055e+07,-3.668429e+07
P_DOP,-1.166010e+06,-1.090792e+06,-9.102942e+05
P_FRP,-1.166010e+06,-1.090792e+06,-9.102942e+05
P_Particulate,NaN,-1.114075e+11,-1.370543e+11


In [ ]:
# As might be expected, increasing the impervious fraction increases flow and consequently the constituents also increase....